# Tarea Dask

## Instrucciones generales
- Esta tarea debe realizarse de manera individual
- Este notebook (resuelto) debe ser subido al github del proyecto en la carpeta de tareas (creen una carpeta dentro de esa carpeta y agreguen su notebook reuelto)
- Fecha límite: Lunes 25 de noviembre de 2024 a las 11:59 p.m
- Deben realizar las cuatro secciones
- Puedes agregar tantas celdas de código y explicaciones como veas necesario, solo manten la estructura general

## Sección 0 Creación y Configuración del cliente de Dask
Ejercicio 0: Configuración del cliente
1. Crea un cliente local de Dask que inicie un clúster en tu máquina.
2. Configura el cliente para que tenga las siguientes características (elige un par de las opciones de trabajadores e hilos):
    - Número de trabajadores: 2 / 4
    - Memoria máxima por trabajador: 1GB
    - Threads por trabajador: 4 / 2
3. Verifica que el cliente esté funcionando correctamente mostrando:
    - Resumen de los trabajadores activos.
    - Dashboard disponible (URL del panel de control de Dask).
    * Tip: Checa los parámetros del cliente que creeaste.

*Nota*: Puedes hacer que corra en el puerto que desees.

In [ ]:
# Tu código va aquí
from dask.distributed import Client

client = Client(
    n_workers = 2, 
    threads_per_worker=4,
    memory_limit='1GB',
)

## Sección 1 Delayed
Ejercicio 1: Procesamiento de datos 

1. Genera datos simulados (por ejemplo, ventas diarias) para 10 sucursales durante 365 días.
    - Cada sucursal debe tener datos generados aleatoriamente para "Ingresos" y "Costos".
    - Utiliza una función para generar los datos simulados.
2. Usa Dask Delayed para calcular:
    - Las ganancias diarias por sucursal.
    - La sucursal con mayor ganancia promedio.
3. Genera un grafo de tareas que visualice estas operaciones y explica por qué elegiste paralelizar de esa forma, genera una visualización del grafo.

In [ ]:
# Tu código va aquí
import dask
import numpy as np
import pandas as pd

def generar_datos(sucursal, num_registros=365):
    np.random.seed(sucursal)  
    data = {
        "sucursal": [sucursal] * num_registros,
        "fecha": pd.date_range(start='2023-01-01', periods=num_registros, freq='D'),
        "Ingresos": np.random.randint(1000, 10000, num_registros), 
        "Costos": np.random.randint(500, 5000, num_registros)
    }
    return pd.DataFrame(data)

def calcular_ganancias(df):
    df['Ganancias'] = df["Ingresos"] - df["Costos"]
    return df

@dask.delayed
def procesar_sucursal(sucursal):
    datos = generar_datos(sucursal)
    datos_con_ganancias = calcular_ganancias(datos)
    ganancia_promedio = datos_con_ganancias['Ganancias'].mean()
    return {
        'sucursal': sucursal,
        'datos': datos_con_ganancias,
        'ganancia_promedio': ganancia_promedio
    }

@dask.delayed
def encontrar_mejor_sucursal(resultados):
    ganancias_promedio = {res['sucursal']: res['ganancia_promedio'] for res in resultados}
    mejor_sucursal = max(ganancias_promedio.items(), key=lambda x: x[1])
    return mejor_sucursal


tareas_sucursales = [procesar_sucursal(sucursal) for sucursal in range(10)]
mejor_sucursal = encontrar_mejor_sucursal(tareas_sucursales)

grafo = mejor_sucursal.visualize(filename='grafo_ventas.png')
resultados = dask.compute(tareas_sucursales, mejor_sucursal)
datos_sucursales, (mejor_sucursal_id, ganancia_promedio) = resultados

print("\nResumen de ganancias por sucursal:")
for resultado in datos_sucursales:
    sucursal = resultado['sucursal']
    ganancia_prom = resultado['ganancia_promedio']
    print(f"Sucursal {sucursal}: ${ganancia_prom:.2f}")

print(f"Sucursal con mayor ganancia promedio: {mejor_sucursal_id}")
print(f"Ganancia promedio: ${ganancia_promedio:.2f}")


# Paralelizado por: 
    # independencia de los datos, sucursal
    # sin dependencias entre el calculo de las ganancias de cada sucursal

## Sección 2 Dask Dataframes
Ejercicio 2: Limpieza y análisis de datos reales

1. Descarga un conjunto de datos masivo (puedes usar la colección de *nycflights* que se encuentra en `data/nycflights/`).
2. Carga los datos en un Dask DataFrame. 
    - Elige adecuadamente el número de particiones (que quepan en memoria de los `workers`)
3. Realiza las siguientes tareas:
    - Limpia los valores faltantes en las columnas `ArrDelay` y `DepDelay`, rellenándolos con la mediana de cada columna.
    - Calcula el retraso promedio (`DepDelay`) por mes y aerolínea.
    - Encuentra el aeropuerto de origen con más vuelos retrasados.

*Nota*: **Evita** convertir el DataFrame a pandas e **intenta** realizar `.compute()` solo cuando sea necesario.

In [ ]:
# Tu código va aquí
import dask.dataframe as dd

df = dd.read_csv('notebooks/data/nycflights/*.csv', npartitions=4)

medianas = df[['ArrDelay', 'DepDelay']].quantile(0.5).compute()
df['ArrDelay'] = df['ArrDelay'].fillna(medianas['ArrDelay'])
df['DepDelay'] = df['DepDelay'].fillna(medianas['DepDelay'])

retrasos_mes_aerolinea = df.groupby(['Month', 'UniqueCarrier'])['DepDelay'].mean()

vuelos_retrasados = df[df['DepDelay'] > 0].groupby('Origin').size()
aeropuerto_mas_retrasado = vuelos_retrasados.nlargest(1)

## Sección 3 Dask Arrays

Ejercicio 3: Procesamiento numérico avanzado

1. Crea un arreglo de 10,000 x 10,000 con valores aleatorios usando Dask Array, utiliza un tamaño de chunks adecuado, ¿es mejor que sean cuadrados?.
2. Realiza las siguientes operaciones:
    - Calcula la suma de cada fila.
    - Encuentra la fila con el valor máximo promedio.
    - Multiplica todo el arreglo por un factor escalar (por ejemplo, 2.5).
3. Divide el arreglo nuevamente en 100 bloques y compara la rapidez.

In [ ]:
# Tu código va aquí
import time
import dask.array as da

def procesar_array(shape, chunks, descripcion):
   start_time = time.time()

   array = da.random.random(shape, chunks=chunks)

   suma_de_filas = array.sum(axis=1)
   promedios_filas = array.mean(axis=1)
   indice_max = promedios_filas.argmax()
   array_multiplicado = array * 2.5

   resultados = {
      'suma_filas': suma_de_filas.compute(),
      'indice_max': indice_max.compute(),
      'muestra_multiplicada': array_multiplicado[0:2, 0:2].compute()
   }
   
   tiempo_total = time.time() - start_time

   return tiempo_total, resultados

shape = (10000, 10000)

# chunks cuadrados
tiempo_cuadrados, resultados_cuadrados = procesar_array(
   shape, 
   chunks=(1000, 1000), 
   descripcion="chunks cuadrados"
)

# 100 bloques
tiempo_cien_bloques, resultados_cien = procesar_array(
   shape, 
   chunks=(100, 100),
   descripcion="100 bloques"
)

print("\nComparación de tiempos:")
print(f"Chunks cuadrados (1000x1000): {tiempo_cuadrados:.2f} segundos")
print(f"100 bloques (100x100): {tiempo_cien_bloques:.2f} segundos")
print(f"\nDiferencia de tiempo: {abs(tiempo_cuadrados - tiempo_cien_bloques):.2f} segundos")
print(f"La configuración {'de chunks cuadrados' if tiempo_cuadrados < tiempo_cien_bloques else 'de 100 bloques'} fue más rápida")

## Sección 4 Futures
Ejercicio 4: Distribución de tareas dinámicas

1. Implementa una función que calcule la raíz cuadrada de una lista de 100,000 números enteros generados aleatoriamente.
2. Divide la lista en 10 partes iguales y usa Dask Futures para calcular la raíz cuadrada de cada parte en paralelo.
3. Recolecta los resultados y calcula:
    - El promedio de todos los números procesados.
    - El tiempo total de ejecución (incluyendo envío y recolección de tareas).
4. Observa como se distribuye la carga en el cliente.

*Nota*: en los ejercicios ya vimos como determinar si ya se cumplío una tarea.

In [ ]:
# Tu código va aquí
import random 
from dask.distributed import as_completed

def calcular_raiz(n):
    return n ** 0.5

numeros = [random.randint(1, 1000) for _ in range(100000)]
futuros = [client.submit(calcular_raiz, num) for num in numeros]

resultados = []
for futuro in as_completed(futuros):
    resultados.append(futuro.result())

promedio = sum(resultados) / len(resultados)

# 4. Mostrar resultados
print("Primeros 10 resultados:", resultados[:10])
print(f"Total de tareas completadas: {len(resultados)}")
print(f"Promedio de las raíces cuadradas: {promedio:.2f}")